# Gollosary and file format specification

* **Difficulty level**: easy
* **Time need to lean**: 10 minutes or less
* **Key points**:

  

## Terminology & Grammar <a id="Terminology_Grammar"></a>

A SoS **script** defines one or more **workflows**, and each workflow consists of one or more **steps**. 

SoS steps are defined by SoS **sections**. A SoS **section** consists of a **header** with one or more step names and optional options. The body of a SoS section consists of optional **comments**, **statements**, **input**, **output**, **depends** directives, **parameter** definitions, and external **task** definition. 


The following terms will be used throughout this documentation:

* **Script**: A SoS script that defines one or more workflows.
* **Section**: A group of statements with a header that defines one or more SoS steps.
* **Workflow**: A sequence of steps that can be executed to complete certain task.
* **Step**: A step of a workflow that performs one piece of the workflow.
* **Target**: Objects that are input or result of a SoS step, which are usually files, but can also be objects such as executable commands and SoS variables.
* **Step options**: Options of the step that assist the definition of the workflow.
* **Step input**: Specifies the input files of the step.
* **Step output**: Specifies the output files and targets of the step.
* **Step dependencies**: Specifies the files and targets that are required by the step.
* **Substep**: A substep consists of all statements after the `input` specification. It can be executed multiple times each with a subset of input files and/or different parameters.
* **Task**: Part or all substep that will be executed outside of SoS, and potentially on a different server. These are usually resource intensive jobs that will take a long time to complete.
* **Action**: SoS or user-defined Python functions. They differ from regular Python functions in that they may behave differently in different running mode of SoS (e.g. ignore when executed in dryrun mode).

More formally defined, the SoS syntax obeys the following grammar, given in extended Backus-Naur form (EBNF):

```
Script         = {comment}, {step};
comment        = "#", text, NEWLINE
assignment     = name, "=", expression, NEWLINE
```

with SoS steps defined as

```
step           = [step_header],
                 {comment}, {{statement}, [input | output | depends ]},
                 [process, NEWLINE, {script} ]
step_header    = "[", step_names, [":", names | options], "]", NEWLINE
parameter      = "parameter", ":", assignment
input          = "input", ":", [expressions], [",", options], NEWLINE
output         = "output", ":", [expressions], [",", options], NEWLINE
depends        = "depends", ":", [expressions], [",", options], NEWLINE
task           = "task", ":",  [options]
action         = func_format | script_format
func_format    = name, "(", [options], ")"
script_format  = name, ":", [options], NEWLINE, script 
step_names     = step_name | step_pattern, ["(", text, ")"],
                 {",", step_name | steo_pattern, ["(", text, ")"]}
names          = name, {",", name}
workflow       = name, ['_', steps], {"+", name, ['_', steps}
assignment     = name, "=", expression, NEWLINW
expressions    = expression, {",", expression}
options        = option, {"," option}
option         = name, "=", expression
step_name      = letter, { letter | digit | "-" | "_" }
step_pattern   = {letter | digit | "*" | "?" | "-" | "_"}
```

Here `name`, `expression` and `statement` are arbitrary [Python 3](http://www.python.org) names, expression and statements with added SoS features.

**SoS is based on the Python 3 (3.6 and above) programming language**. If you are unfamiliar with Python, you can learn some basics of Python, usually in less than half a day, by reading some Python tutorials (e.g. [the official python tutorial](https://docs.python.org/3/tutorial/)). This [short introduction](https://docs.python.org/3/tutorial/introduction.html) is good enough for you to get started with SoS.

SoS adds the following syntax to standard Python syntax: 

| syntax | Example | Main Usage |
|--------|---------|-------|
|**Script format of function call**| `R: expand=True`<br>`  data <- read.csv("{_input}")`| Verbatim inclusion of scripts with optional indentation and string interpolation |
|**Section specification**| `[align_20]` | Define steps of workflows |
|**SoS statements**||Direct execution of steps|
|| **`parameter`**`: cutoff=10` | Obtain option from command line or workflow caller |
|| **`input`**`: fastq_files` | Specify input files of a step |
|| **`output`**`: f"{_input}.idx"` | Specify output files of a step |
|| **`depends`**`: hg19_fa` | Specify step dependencies |
|| **`task`**`: queue='cluster'` | Specify external tasks |


This chapter demonstrates general features such as string interpolation, section specification, parameter specification, and specification of workflows. Please refer to chapters [SoS Step](SoS_Step.html) and [Remote Exection](Remote_Execution.html) for more details on `input`, `output`, `depends`, and `task` statements.

### Using `sos_targets`

Whereas you can always use `step_input`, `_input` etc in scripts (format-string) easily, using them outside of scripts is a bit tricker. Let us check a few common usages and see how to use `sos_targets` in SoS.

First, it is a common practice to generate another file with a slightly different name. For example, in the following script, `step_input` is a `sos_targets` with two filenames, `_input` is a `sos_targets` with one filename, and we need to generate `test.bam.bai` from input file `test.bam`. In this particular case, you can append the `.bai` extension directly to `_input` using the `+` operator, by passing the `+` operation to the first `file_target` of `_input`:

In [46]:
%sandbox
!touch test1.bam test2.bam

input_files = ['test1.bam', 'test2.bam']
input: input_files, group_by=1
output: _input + '.bai'

print(f"Generating {_output} from {_input}")
_output.touch()

Generating /private/var/folders/ys/gnzk0qbx5wbdgm531v82xxljv5yqy8/T/tmpenoojx6z/test1.bam.bai from test1.bam
Generating /private/var/folders/ys/gnzk0qbx5wbdgm531v82xxljv5yqy8/T/tmpenoojx6z/test2.bam.bai from test2.bam


If the extensions are different, you can use the `with_suffix` function of `pathlib.Path`, again by calling the `with_suffix` function of its only element `_input[0]`:

In [47]:
%sandbox
!touch test1.bam test2.bam

input_files = ['test1.bam', 'test2.bam']
input: input_files, group_by=1
output: _input.with_suffix('.bai')

print(f"Generating {_output} from {_input}")
_output.touch()

Generating test1.bai from test1.bam
Generating test2.bai from test2.bam


You can also use the `:n` formatter to retrieve the part of `_input` without extension, and add `.bam.bai` to it as follows:

In [48]:
%sandbox
!touch test1.bam test2.bam

input_files = ['test1.bam', 'test2.bam']
input: input_files, group_by=1
output: f"{_input:n}.bam.bai"

print(f"Generating {_output} from {_input}")
_output.touch()

Generating test1.bam.bai from test1.bam
Generating test2.bam.bai from test2.bam


Note that the result of `_input + '.bai'` is a `path` and `f'{_input:n}.bam.bai'` is a `str` but both will be converted to `_output` with type `sos_targets`. You can use either style although `:u`, `:b` etc can be shorter than `expanduser()` and `.name`, especially when you combine multiple formatters to extract certain parts of a `path`.

Second, because `path`, `path`, and even `sos_targets` supports the `os.PathLike` interface, you can use them as filenames directly in SoS (Python). For example, you can pass `sos_targets` with one element to usual `os.path` functions such as `getsize` and open it directly with the `open` function:

In [49]:
input: "SoS_Syntax.ipynb"
print(f"Size of {repr(_input)} is {os.path.getsize(_input)}.")
print(f"The first two lines of {repr(_input)} are:")

with open(input, 'r') as myself:
    print(myself.readline())
    print(myself.readline())

Size of [file_target('SoS_Syntax.ipynb')] is 1062666.
The first two lines of [file_target('SoS_Syntax.ipynb')] are:
{

 "cells": [



However, although Python standard libraries supports `os.PathLike` interface, third party modules might or might not work correctly in this day so it can be safer to use `str(input)` to convert the paths to `str` before passing them to these functions.

## Native SoS file format <a id="Native_SoS_file_format"></a>

A sos script can be defined in a plain text file. A `.sos` suffix is recommended but not required. A SoS script consists of **sections** that define **steps** of one or more **workflows**.

A SoS script usually starts with lines

```python
#!/usr/bin/env sos-runner
#fileformat=SOS1.0
```

The first line allows the script to be executed by command `sos-runner` if it is executed as an executable script. The second line tells SoS the format of the script. The `#fileformat` line does not have to be the first or second line but should be in the first comment block. The latest version of SOS format is assumed if no format line is present so it is a good practice to specify version of file format to make sure the script is interpreted correctly.

### Global sections and default variables <a id="Global_sections_and_default_variables"></a>

Global sections can be defined without section header in a `.sos` file as statements before any other section, and as a regular section with header `[global]`. The global sections are the only section that appear multiple times in a SoS script.

Definitions in the global section are shared by all sections so it is usually used to define global variables and parameters. SoS implicitly defines the following variables in the global section:

* **`SOS_VERSION`**: version of SoS interpreter.
* **`CONFIG`**: configurations read from site, hosts, global, local and user specific configuration files.

### SoS Sections <a id="SoS_Sections"></a>

A SoS section is marked by a section header in the format of

```
[names: options]
```

The header should start with a `[` from the beginning of a line and end with a `]`. It can contain one or more names with optional description (for each step) and section options (for all steps defined in the section).

Section names of a section follow the following rules:

| Format | Example | Usage |
|--------|---| -------|
| **`name_index`** |`human_10`|Defines step `index` of workflow `name`. Here `name` can be any name with alpha-numeric characters and `-` and `_`. `index` should be a non-negative number.|
| **`name`** |`update-website`| Section name without index is equivalent to `name_0` |
| **`index`** |`10`| Section name without workflow name is equivalent to `default_index`|
| **`pattern_index`** |`*_0`, `human*_10`| Equivalent to step `index` of all matching workflows defined in the script. The `pattern` should follow [Unix filename matching](https://docs.python.org/2/library/fnmatch.html)|
| **`stepname (desc)`**| `10 (align)`| Optional short description can be used to describe the goal of the step|
| **`name1,name2,...`** |`human_10,mouse_10`| Comma separated names define multiple steps for one or more workflows|

A SoS step accepts the following options:

| Name | Example | Usage |
|------|--------|---------|
|**`skip`** | `skip`<br>`skip=False` | Always or conditionally skip a step, `expr` should be evaluated as `True` or `False`|
|**`shared`**|`shared='K'`<br>`shared={'K': 'output[0]'}`  | Variable that will be made available to the workflow after completion of the step |
|**`provides`**|`provides='filename'`<br>`provides=executable('fastqc')`<br>`provides='{filename}.bam.gz'`<br>`provides=['A1.txt', 'A2.txt']`| Targets that will be generated after the completion of the step. This option turns the step to an auxiliary step that will be executed when the provided target is needed. |

Please refer to section [SoS Step](SoS_Step.html) for more details on these options. 

A section can have arbitrary Python statements and SoS-specific statements that define the input, output, and dependent targets, and external tasks of the step.

|Keyword|Examples|Usage|
|-------|-------|-----|
|**`input`**| (unspecified)<br> `input: 'test1.txt', 'text2.txt'`<br>`input: '*.txt'` <br> `input: dynamic('*.txt')`| Input targets of the step, which can be derived from previous step (if unspecified), determined in advanced or only before the step is executed. |
|**`depends`**| `depends: 'hg19.fa'`<br>`depends: f"{_input}.idx"`<br>`depends: R_library('ggplot2')`| Step depends on a specified files or targets to execute |
|**`output`**| (unspecified) <br>`output: 'output.bam'`<br>`output: f'{_input}.idx'`| Step has no or specified defined output that can be derived from input files |
|**`task`**|`task:`| Statements following the statement will be executed as an external task|

These statements accept a large number of options to specify how input, output targets are processed and how external tasks are executed. Please refer to section [SoS step](SoS_Step.html) for more details about different types of steps and step options, and section [Remote Exection](Remote_Execution.html) on topics related to external tasks.

As an example for a complete script, the following script defines a workflow with a global section without header, and a workflow `gff` with two steps

```sos
#!/usr/bin/env sos-runner
#fileformat=SOS1.0

local_resource = '~/Resource/'
data_dir       = '~/Data/bams/'
resource_dir   = f'{local_resource}/resources/hg19/Ensembl/Genes'

# samples to be processed
parameter: samples = ['s312', 's315', 's312a', 's315a']

[gff_0]
# download gene models from the MISO website
output: f'{resource_dir}/Home_sapiens.GRCh37.65.gff.zip'
download: dest_dir=resource_dir, decompress=True
    http://genes.mit.edu/burgelab/miso/annotations/gene-models/Homo_sapiens.GRCh37.65.gff.zip

[gff_1]
# Index gtf file using index_gff
output: f'{resource_dir}/{hg19_gff_index}/genes.gff'
task:   working_dir=resource_dir
run:    docker_image='mdabioinfo/miso:latest'
    rm -rf {hg19_gff_index}
    index_gff --index {hg19_gff_file} {hg19_gff_index}
```